# Knapsack Problems

## Abstract

## Introduction

## Experiment

In [ ]:
%pip install numpy matplotlib qiskit qiskit_ibm_runtime tqdm

In [ ]:
# Import necessary libraries.

import copy
import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from tqdm import tqdm

# Global parameters.

MAX_GENERATION = 100
MAX_REPEATS = 5
MIN_ITEMS = 10
MAX_ITEMS = 127
STEPS = 5
SHOTS = 10_000

# Get the backend ready.

token = ''
service = QiskitRuntimeService(channel='ibm_quantum', token=token)
backend = service.backend('ibm_rensselaer')
pass_manager = generate_preset_pass_manager(optimization_level=3, backend=backend)

In [ ]:
# Measure the quantum circuit.
def make(qc: QuantumCircuit) -> list:
    qc.measure_all()
    trans_qc = pass_manager.run(qc)

    sampler = Sampler(mode=backend)
    job = sampler.run([trans_qc], shots=SHOTS)
    result = job.result()
    counts = result[0].data.meas.get_counts()

    binary = max(counts, key=counts.get)
    binary_list = [int(bit) for bit in binary]
    
    return binary_list

# Initializing the circuit with Hadamard gates for superposition.
def initialize(num_items: int) -> QuantumCircuit:
    qc= QuantumCircuit(num_items)
    for i in range(num_items):
        qc.h(i)
    
    return qc

# Repair the knapsack.
def repair(weights: list, capacity: float, binary: list, num_items: int) -> list:
    w = np.array(weights)
    x = np.array(binary)
    i, j = 0, 0

    overfilled = False
    if w @ x > capacity:
        overfilled = True

    while overfilled:
        i = np.random.randint(0, num_items)
        x[i] = 0
        if w @ x <= capacity:
            overfilled = False

    while not overfilled:
        j = np.random.randint(0, num_items)
        x[j] = 1
        if w @ x > capacity:
            overfilled = True

    x[j] = 0

    return x

# Compute profit.
def evaluate(values: list, binary: list) -> float:
    p = np.array(values)
    x = np.array(binary)

    return p @ x

# Get rotation angles.
def get_thetas(x: list, b: list, fx: float, fb: float, num_items: int) -> list:
    geq = fx >= fb

    thetas = []
    for i in range(num_items):
        if x[i] == 0 and b[i] == 0 and not geq:
            thetas.append(0.0)
        elif x[i] == 0 and b[i] == 0 and geq:
            thetas.append(0.0)
        elif x[i] == 0 and b[i] == 1 and not geq:
            thetas.append(0.0)
        elif x[i] == 0 and b[i] == 1 and geq:
            thetas.append(2 * 0.05 * np.pi)
        elif x[i] == 1 and b[i] == 0 and not geq:
            thetas.append(2 * 0.01 * np.pi)
        elif x[i] == 1 and b[i] == 0 and geq:
            thetas.append(2 * 0.025 * np.pi)
        elif x[i] == 1 and b[i] == 1 and not geq:
            thetas.append(2 * 0.005 * np.pi)
        elif x[i] == 1 and b[i] == 1 and geq:
            thetas.append(2 * 0.025 * np.pi)
        else:
            raise RuntimeError("There must a typo in one of these if statements.")

    return thetas

# Updating qubit's angle by thetas.
def update(qc: QuantumCircuit, thetas: list, num_items: int) -> QuantumCircuit:
    for i in range(num_items):
        qc.ry(thetas[i], i)

    return qc

# Compare lists eleemnt-wise and length-wise.
def equal_lists(list_a: list, list_b: list) -> bool:
    if len(list_a) != len(list_b):
        return False

    return all(a == b for a, b in zip(list_a, list_b))

# Perform genetic quantum algorithm.
def gqa(weights: list, values: list, capacity: float, num_items: int) -> tuple:
    t = 0
    count_repeat = 0

    qc_accumulated = initialize(num_items)
    qc_measured = copy.deepcopy(qc_accumulated)
    binary = make(qc_measured)
    
    best_solution = repair(weights, capacity, binary, num_items)
    best_profit = evaluate(values, best_solution)
    count_update = 1

    # Try to prevent spamming requests by detecting repeats.
    while t < MAX_GENERATION and count_repeat < MAX_REPEATS:
        t += 1

        qc_measured = copy.deepcopy(qc_accumulated)
        binary = make(qc_measured)
        solution = repair(weights, capacity, binary, num_items)

        profit = evaluate(values, solution)
        thetas = get_thetas(solution, best_solution, profit, best_profit, num_items)
        qc_accumulated = update(qc_accumulated, thetas, num_items)

        if equal_lists(solution, best_solution):
            count_repeat += 1
        else:
            if profit > best_profit:
                best_profit = profit
                best_solution = solution
                count_update += 1
            count_repeat = 0

    return (best_profit, best_solution, count_update)

## Discussion

## Conclusion

## References